In [2]:
import ll
import rich

ll.pretty()

In [3]:
import datasets
import torch
from torch_geometric.data import Data

dataset = datasets.load_dataset("nimashoghi/mptrj", split="val")
dataset.set_format("torch")


def to_pyg(data_dict):
    return Data(
        pos=data_dict["positions"],
        cell=data_dict["cell"].unsqueeze(0),
        natoms=data_dict["num_atoms"],
        tags=torch.zeros_like(data_dict["numbers"], dtype=torch.long),
    )


rich.print(to_pyg(dataset[0]))

[15:25:03] INFO     PyTorch version 2.2.1 available.                                                   ]8;id=952504;file:///opt/conda/envs/jmp-peft/lib/python3.11/site-packages/datasets/config.py\config.py]8;;\:]8;id=932175;file:///opt/conda/envs/jmp-peft/lib/python3.11/site-packages/datasets/config.py#58\58]8;;\

Data(pos=[28, 3], cell=[1, 3, 3], natoms=28, tags=[28])

In [4]:
import torch

from jmppeft.utils.radius_graph import radius_graph_pbc

data = to_pyg(dataset[0])
edge_index, cell_offsets, num_neighbors = radius_graph_pbc(
    radius=6.0,
    max_num_neighbors_threshold=30,
    pos=data.pos,
    cell=data.cell,
    n_atoms=data.natoms,
)

rich.print(
    {
        "edge_index": edge_index,
        "cell_offsets": cell_offsets,
        "num_neighbors": num_neighbors,
    }
)

[15:25:45] CRITICAL Type checking the following modules: ('jmppeft',)                               ]8;id=687553;file:///workspaces/ll/src/ll/typecheck.py\typecheck.py]8;;\:]8;id=516439;file:///workspaces/ll/src/ll/typecheck.py#65\65]8;;\

{
    'edge_index': tensor[2, 840] i64 n=1680 (13Kb) x∈[0, 27] μ=13.796 σ=8.082,
    'cell_offsets': tensor[840, 3] n=2520 (9.8Kb) x∈[-1.000, 1.000] μ=0.001 σ=0.531,
    'num_neighbors': tensor[1] i64 [840]
}

In [19]:
import torch

from jmppeft.utils.radius_graph import radius_graph_pbc
from torch_geometric.data import Batch

data = Batch.from_data_list([to_pyg(dataset[0]), to_pyg(dataset[0])])
data.edge_index, data.edge_cell_offsets, data.num_neighbors = radius_graph_pbc(
    radius=6.0,
    max_num_neighbors_threshold=30,
    pos=data.pos,
    cell=data.cell,
    n_atoms=data.natoms,
)

rich.print(
    data,
    {
        "edge_index": data.edge_index,
        "cell_offsets": data.edge_cell_offsets,
        "num_neighbors": data.num_neighbors,
    },
)

size_so_far = 0
for bsz_i in range(data.batch.max().item() + 1):
    node_mask = data.batch == bsz_i
    edge_mask = node_mask[data.edge_index[0]]
    assert (edge_mask == node_mask[data.edge_index[1]]).all()
    rich.print(
        {
            "edge_index": data.edge_index[:, edge_mask] - size_so_far,
            "cell_offsets": data.edge_cell_offsets[edge_mask],
            "num_neighbors": data.num_neighbors,
        },
    )

    size_so_far += node_mask.sum().item()

DataBatch(pos=[56, 3], cell=[2, 3, 3], natoms=[2], tags=[56], batch=[56], ptr=[3], edge_index=[2, 1680], 
edge_cell_offsets=[1680, 3], num_neighbors=[2])
{
    'edge_index': tensor[2, 1680] i64 n=3360 (26Kb) x∈[0, 55] μ=27.796 σ=16.167,
    'cell_offsets': tensor[1680, 3] n=5040 (20Kb) x∈[-1.000, 1.000] μ=0.001 σ=0.531,
    'num_neighbors': tensor[2] i64 μ=840.000 σ=0. [840, 840]
}

{
    'edge_index': tensor[2, 840] i64 n=1680 (13Kb) x∈[0, 27] μ=13.796 σ=8.082,
    'cell_offsets': tensor[840, 3] n=2520 (9.8Kb) x∈[-1.000, 1.000] μ=0.001 σ=0.531,
    'num_neighbors': tensor[2] i64 μ=840.000 σ=0. [840, 840]
}

{
    'edge_index': tensor[2, 840] i64 n=1680 (13Kb) x∈[0, 27] μ=13.796 σ=8.082,
    'cell_offsets': tensor[840, 3] n=2520 (9.8Kb) x∈[-1.000, 1.000] μ=0.001 σ=0.531,
    'num_neighbors': tensor[2] i64 μ=840.000 σ=0. [840, 840]
}